In [13]:
%load_ext autoreload
%autoreload 2
from simuq import QSystem
from simuq import Qubit
import numpy as np
from simuq.ionq.ionq_provider import IonQProvider
ionqp = IonQProvider(from_file="../../../ionq_API_key")

def GenQS(n=12, p=3):
    qs = QSystem()
    ql = [Qubit(qs) for i in range(n)]
    link = [(i, (i + 1) % n) for i in range(n)]
    if p==3:    
        parameter_list = np.array(
        [
            0.5702193 * 2,
            -0.58631086,
            0.85160685 * 2,
            -1.7058538,
            0.29468536 * 2,
            -1.132814,
        ]
    )/2
    if p==2:
        parameter_list = np.array([-3.8034,1.2438, -1.2467,-2.4899])/2
    if p==1:
        parameter_list = np.array([-0.7854,-2.3562])/2

    for i in range(p):
       
        h = 0
        for q0, q1 in link:
            h += parameter_list[2 * i] * ql[q0].Z * ql[q1].Z
        qs.add_evolution(h, 1)
        h = 0
        for q0 in range(n):
            h += parameter_list[2 * i+1] * ql[q0].X
        qs.add_evolution(h, 1)
    return qs

def calc_cut(count):
    cut=0
    n_shots=sum(count.values())
    for key in count:
        cut+=count[key]*calc_weight(key)/n_shots
    return cut
def calc_weight(key):
    cut=0
    for i in range(len(key)):
        if key[i]!= key[(i+1)%len(key)]:
            cut+=1
    return cut

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from simuq.ionq.ionq_api_circuit import IonQAPICircuit

state_prep = IonQAPICircuit(12)
for i in range(12):
    state_prep.gpi2(i, np.pi / 2)

for p in [2,3]:
    qs = GenQS(12, p)
    ionqp.compile(
        qs,
        backend="aria-1",
        aais="heisenberg",
        tol=0.01,
        trotter_num=4,
        state_prep=state_prep,
    )
    ionqp.run(shots=1024)
    # result = bp.results()
    # print(result)
    # print(calc_cut(result))

{'id': '096940b6-2f0f-4027-8a20-e46c8b0ce97a', 'status': 'ready', 'request': 1696879149}
{'id': 'b5f7e154-74c5-4bd1-9eef-2b76ef5ddf82', 'status': 'ready', 'request': 1696879151}


In [ ]:
calc_cut(ionqp.results(job_id="2eb797d2-d655-40dd-a004-68f856aca728"))

In [7]:
calc_cut(ionqp.results(job_id="096940b6-2f0f-4027-8a20-e46c8b0ce97a"))

9.999709411997685

In [8]:
calc_cut(ionqp.results(job_id="b5f7e154-74c5-4bd1-9eef-2b76ef5ddf82"))

10.499928995991466

In [ ]:
{'id': '607989f8-2a73-4a75-8412-2993f332f40a', 'status': 'ready', 'request': 1696878112}
{'id': 'db61b0cf-0e9b-4630-a45e-6525733a615f', 'status': 'ready', 'request': 1696878113}
{'id': '655ed670-9f93-4ea4-81ee-3eeabbf8fb81', 'status': 'ready', 'request': 1696878115}